# **Image Retrieval By Caption with GUI**

## **1. Introduciton**
Retrieving images by queries has always been a relevant task. In this project, we are going to explore one way of completing this task using deep learning methods. We are even going to incorporate everything into a running application, so users can play with the model more easily. The overall method is the following: We are going to create vector embeddings for the images and their captions respectively. The two encoders are going to learn to make similar embeddings for the right image-caption pairs and this way, the models are going to capture semantics. After this, when the user gives a prompt, the text encoder is going to create an embedding for it and look for the closest images to it. In the end, the model is packed into a streamlit application so everyone can play with it.

The application is in the "app" folder in the main directory of the project and has to be run with the following command:

`streamlit run main.py`

## **2. Dataset**
The dataset we are going to use here is called, **Flickr 8K**. It consists of **8091** images, each with **5** unique captions. The data is split into a train, test and validation set in the proportion **80/10/10**. PyTorch datasets are created for each of the splits, where several transformations are performed (Resizing to 256x256, Center-cropping to 224x224 and Normalizing) and then all of the images are kept in a tensor object in the dataset. For training, upon calling `__getitem__`, the dataset returns an image with the corresponding index and one random caption out of 5 possible. This helps the model train better.

## **3. Model setup and training**
### **3.1 Model setup**
#### **3.1.1 Image Encoder**
The image encoder consists of a pre-trained **EfficientnetB0** model with its classification head removed. The embeddings it creates are then passed through a linear layer to project them onto **256** dimensions, which is the desired embedding size.
#### **3.1.2 Text Encoder**
The text encoder consists of a pre-trained **BERT** model. Its embeddings are again projected onto **256** dimensions through a single linear layer.
### **3.2 Training process**
The loss function used in this project is called **NT-Xent**. Experiments showed, that the best temperature value is **0.1**.

During the first stage of the training, the backbone models were freezed and only the last linear layers for both of the encoders were trained. The **AdamW** optimizer with a weight decay of **0.001** was used across the board. The first 20 epochs were done with a learning rate of **0.01**, then 10 more epochs were done with a learning rate of **0.001**. At this point, the model seemed to begin overfitting a little so the backbone models were unfreezed. 20 more epochs were conducted with all of the parameters unfrozed with the learning rate set to **0.00001**. This showed a little improvement and clear signs of overfitting. For the sake of the experiment, 20 more epochs were conducted with no improvement, so the best models is the one, saved at epoch 50. Also, experiments were conducted with only the last few layers of each of the backbone models being unfrozen, although the performance was worse.


For more detailed information with the code and experiments one can refer to `model_training.ipynb` in the "notebooks" folder.

## **4. Testing end evaluating performance**
For the evaluation, two metrics were used: **Recall@k** and **Precision@k**.
### **4.1 Recall@K**
**Recall@K** is a metric, that calculates how many of the relevant images are shown in the top K results. It's a modified vecsion of the classic **recall** metric, where we now look at the top K results instead of just one. A higher **Recall@K** value means, that the model collectly retrieves the relevant items, while a lower value means, that the model misses out on important images.
### **4.2 Precision@K**
**Precision@K** calculates what proportion of the retrieved results are relevant. Again, it's a modified version of the classic precision metric. The higher the value is, the better the model is at retrieving only relevant information.
### **4.3 Results**
The model did not show promising results for these metrics. For both of them, the results are below the acceptable levels. At least the results for the training data are not very far from the testing data so the model didn't overfit. Here are the plots with the metrics: 
![Here is the plot with the Recall@K values:](plots/recall_k.png)
![And here is the plot with the Precision@K values:](plots/precision_k.png)


After playing with the model in the GUI, the images seemed kind of random and unrelated to the query. This means, that the model didn't learn the data properly.

More detailed experiments and information can be seen in `model_evaluation.ipynb` in the "notebooks" folder.

## **5. Conclusion**
Although the performance of the model is far from impressive, there is much room for improvement. Future work can include more intelligent fine-tuning, data augmentation, perhaps even a complete change of the "backbone" models. Also, the application can be modified to get the best captions it has, based on an image, given by the user.

## **6. Citations**
1. Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. In Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies (NAACL-HLT) (pp. 4171-4186).
2. Ågren, W. (2022). The NT-Xent Loss Upper Bound. arXiv preprint arXiv:2205.03169.
3. Tan, M., & Le, Q. V. (2019). EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks. In Proceedings of the 36th International Conference on Machine Learning (pp. 6105-6114). PMLR.
4. Hodosh, M., Young, P., & Hockenmaier, J. (2013). Framing Image Description as a Ranking Task: Data, Models and Evaluation Metrics. Journal of Artificial Intelligence Research, 47, 853-899.